In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# =====================
# Setup MLflow
# =====================
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("iris-comparison-different")

# =====================
# Load dataset (with noise to make task harder)
# =====================
iris = load_iris()
X, y = iris.data, iris.target

# Add Gaussian noise so models behave differently
rng = np.random.RandomState(42)
X_noisy = X + rng.normal(0, 0.3, X.shape)

# Smaller training set = harder problem
X_train, X_test, y_train, y_test = train_test_split(
    X_noisy, y, test_size=0.35, random_state=7, stratify=y
)

# =====================
# Define models
# =====================
models = {
    "Logistic Regression": LogisticRegression(max_iter=100, solver="liblinear", C=0.2),
    "Random Forest": RandomForestClassifier(n_estimators=20, max_depth=2, random_state=42),
    "SVM": SVC(kernel="rbf", probability=True, C=1.5, gamma=0.5, random_state=42)
}

# Directory for results
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# Store results for comparison
results = []

# =====================
# Train & Log models
# =====================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name) as run:
        print(f"\n🚀 Training and logging: {model_name}")

        # Train
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
   # ---- Metrics ----
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")

        print(f"{model_name} - Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.log_metric("f1_score", f1)

        # ---- Parameters ----
        if isinstance(model, LogisticRegression):
            mlflow.log_param("C", model.C)
            mlflow.log_param("solver", model.solver)
        elif isinstance(model, RandomForestClassifier):
            mlflow.log_param("n_estimators", model.n_estimators)
            mlflow.log_param("max_depth", model.max_depth)
        elif isinstance(model, SVC):
            mlflow.log_param("kernel", model.kernel)
            mlflow.log_param("C", model.C)
            mlflow.log_param("gamma", model.gamma)

        # ---- Confusion Matrix ----
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=iris.target_names,
                    yticklabels=iris.target_names)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name}")

        plot_path = os.path.join(results_dir, f"{model_name}_confusion_matrix.png")
        plt.savefig(plot_path)
        plt.close()

        mlflow.log_artifact(plot_path, artifact_path="plots")
  # ---- Log Model ----
        mlflow.sklearn.log_model(model, artifact_path="model")

        # Save results for comparison
        results.append({
            "model_name": model_name,
            "run_id": run.info.run_id,
            "accuracy": acc,
            "model": model
        })

print("\n🎯 All models trained and logged in MLflow.")

# =====================
# Select Best Model & Register
# =====================
best_model = max(results, key=lambda x: x["accuracy"])
print(f"\n🏆 Best Model: {best_model['model_name']} (Accuracy={best_model['accuracy']:.3f})")

# Register in MLflow Model Registry
model_uri = f"runs:/{best_model['run_id']}/model"
mlflow.register_model(model_uri, "IrisBestModel")

print("\n✅ Best model registered in MLflow Model Registry as 'IrisBestModel'")

2025/09/18 10:37:14 INFO mlflow.tracking.fluent: Experiment with name 'iris-comparison-different' does not exist. Creating a new experiment.



🚀 Training and logging: Logistic Regression
Logistic Regression - Accuracy: 0.774, Precision: 0.823, Recall: 0.777, F1: 0.755


2025/09/18 10:37:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 10:37:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/676235388605344173/runs/a4bc44428808445d95b9ac4a0cf917b2
🧪 View experiment at: http://localhost:5000/#/experiments/676235388605344173

🚀 Training and logging: Random Forest
Random Forest - Accuracy: 0.849, Precision: 0.847, Recall: 0.847, F1: 0.847


2025/09/18 10:37:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 10:37:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

2025/09/18 10:37:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 10:37:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'IrisBestModel'.
2025/09/18 10:37:42 WARNING mlflow.tracking._model_registry.fluent: Run with id 83063f23c16b4040a061d3d76acc195f has no artifacts at artifact path 'model', registering model based on models:/m-5f96217921944b169980d38feb9077cd instead


🏃 View run SVM at: http://localhost:5000/#/experiments/676235388605344173/runs/83063f23c16b4040a061d3d76acc195f
🧪 View experiment at: http://localhost:5000/#/experiments/676235388605344173

🎯 All models trained and logged in MLflow.

🏆 Best Model: SVM (Accuracy=0.906)


2025/09/18 10:37:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisBestModel, version 1



✅ Best model registered in MLflow Model Registry as 'IrisBestModel'


Created version '1' of model 'IrisBestModel'.
